In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip '/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip' >&NULL
!unzip '/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip' >&NULL

In [ ]:
os.listdir('/kaggle/working/')

In [ ]:
train_dir="train"
test_dir="test"

In [ ]:
from glob import glob
import random

train_list = glob(os.path.join(train_dir,"*"))
test_list = glob(os.path.join(test_dir,"*"))
test_list=sorted(test_list)
random.shuffle(train_list)
print( "Training Images:",len(train_list) )
print( "Testing Images:",len(test_list) )

In [ ]:
train_list[:4],test_list[:4]

In [ ]:
def check_data(image_list):
    labels=['cat','dog']
    labels_count=[0,0]
    for i in range(len(image_list)):
        image_path=image_list[i]
        image_label=image_path.rsplit('/',1)[1]
        image_label=image_label.split('.',1)[0]
        if image_label not in labels:
            print("error in :",image_path)
            continue
        labels_count[labels.index(image_label)]+=1
    for i in range(len(labels)):
        print("Images with label \"",labels[i],"\":",labels_count[i])
        

print("Training data:\n")        
check_data(train_list)

In [ ]:
import PIL

PIL.Image.open("test/9760.jpg")

In [ ]:
labels=['cat','dog']
label2idx={label:i for i,label in enumerate(labels)}
idx2label={i:label for i,label in enumerate(labels)}

In [ ]:
import tensorflow as tf
# build a lookup table
tensor_label_to_idx = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(labels),
        values=tf.constant([i for i in range(len(labels))]),
    ),
    default_value=tf.constant(-1),
)

# now let us do a lookup
input_tensor = tf.constant(['dog','cat'])
out = tensor_label_to_idx.lookup(input_tensor)
print(out)

In [ ]:
# label2idx['dog']

# tf.compat.v1.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()
input_size=227
# input_size=256
tf.executing_eagerly()

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import albumentations as A


def read_data(img_path):
    
    label=tf.strings.split(img_path,"/")[-1]   # asdsada,cat.2324.jpg 
    label=tf.strings.split(label,".")[0]    # cat, 2324.jpg 
    
    label=tensor_label_to_idx.lookup(label)
    label=K.expand_dims(label,axis=-1)
#     label=tf.one_hot(label,2)
    
    image=tf.io.read_file(img_path)
    image=tf.image.decode_image(image,expand_animations=False)
    image=tf.image.resize(image,[input_size,input_size])
    image/=255
    image.set_shape([input_size,input_size,3])
    
    return image,label

def augment(image,label):
    
    def f(img):
        img*=255
        img=img.astype('uint8')
        # transforms = A.Compose([
        #         A.Rotate(limit=40),
        #         A.RandomBrightness(limit=0.1),
        #         A.HorizontalFlip(),
        #     ])
        transforms = A.Compose([
            A.Rotate(limit=40),
            A.RandomBrightness(limit=0.1),
            A.JpegCompression(quality_lower=85, quality_upper=100, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.RandomContrast(limit=0.2, p=0.5),
            A.HorizontalFlip(),
        ])
        aug_img = transforms(image=img)['image']
        aug_img=aug_img/255
        return aug_img.astype('float32')
        
    image=tf.numpy_function(f,[image],tf.float32)
    return image,label

def tf_dataset(img_list,batch_size=32,aug=False):
    
    dataset=tf.data.Dataset.from_tensor_slices(img_list)
    dataset=dataset.map(read_data)
    if aug: dataset=dataset.map(augment)
    dataset=dataset.shuffle(1000)
    dataset=dataset.batch(batch_size)
    dataset=dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

In [ ]:
ds=tf_dataset(train_list,aug=True)
for batch in ds.take(2):
    x=batch[0].numpy()
    print(batch[0].shape,batch[1].shape)
del ds

In [ ]:
np.min(batch[0]),np.max(batch[0])

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15,20))
for i in range(32):
    fig.add_subplot(7,5,i+1)
    plt.axis('off')
    plt.imshow((x[i]*255).astype(np.uint8))
plt.show()

# models

## model 1

In [ ]:
from tensorflow.keras import layers

def modelv1():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)  #  [(d-f+2p)/s]  + 1 :252,252,32
    x=layers.MaxPooling2D((2,2))(x) # 126,126,32
    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid')(x)
    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.Conv2D(128,(3,3),activation='relu',padding='valid')(x)
    x=layers.Conv2D(128,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.Flatten()(x)
    x=layers.Dense(128,activation='relu')(x)
    x=layers.Dense(64,activation='relu')(x)
    x=layers.Dense(32,activation='relu')(x)
    y=layers.Dense(1,activation='sigmoid')(x)



    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model")
    return model

# model 2

In [ ]:
from tensorflow.keras import layers

def modelv2():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.Flatten()(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu')(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(32,activation='relu')(x)
    x=layers.BatchNormalization()(x)
    y=layers.Dense(1,activation='sigmoid')(x)



    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model_v2")
    return model

Model 3 (light weight)

In [ ]:
from tensorflow.keras import layers

def modelv3():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid')(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.Flatten()(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu')(x)
    x=layers.Dense(32,activation='relu')(x)
    y=layers.Dense(1,activation='sigmoid')(x)



    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model_v3")
    return model

# model 4 (model 2 with he initialization)

In [ ]:
def modelv4():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.BatchNormalization()(x)
    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((2,2))(x)
    x=layers.Flatten()(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    y=layers.Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotUniform())(x)



    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model_v4")
    return model

In [ ]:
def modelv4_overlapping_pooling():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.Flatten()(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    y=layers.Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotUniform())(x)

    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model_v4_op_no_bn")
    return model

In [ ]:
def modelv4_overlapping_pooling_max():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x_input) # [(d-f+2p)/s]  + 1 :254,254,32
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(32,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(64,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2D(128,(3,3),activation='relu',padding='valid',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.MaxPooling2D((3,3),2)(x)
    x=layers.Flatten()(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    x=layers.Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(x)
    x=layers.BatchNormalization()(x)
    y=layers.Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotUniform())(x)

    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="cat_dog_model_v4_op_no_bn")
    return model

In [ ]:
def LeNet5():
    x_input=layers.Input((input_size,input_size,3))
    x=layers.Conv2D(6,(5,5),strides=1,activation='tanh')(x_input)
    x=layers.AveragePooling2D((2,2),strides=2)(x)
    x=layers.Conv2D(16,(5,5),strides=1,activation='tanh')(x)
    x=layers.AveragePooling2D((2,2),strides=2)(x)
    x=layers.Flatten()(x)
    x=layers.Dense(120,activation='tanh')(x)
    x=layers.Dense(84,activation='tanh')(x)
    y=layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="LeNet5")
    return model

In [ ]:
def AlexNet():
    x_input=layers.Input((227,227,3))
    x=layers.Conv2D(96,(11,11),strides=4,activation='relu')(x_input)
    x=layers.MaxPooling2D((3,3),strides=2)(x)
    x=layers.Conv2D(256,(5,5),padding='same',activation='relu')(x)
    x=layers.MaxPooling2D((3,3),strides=2)(x)
    x=layers.Conv2D(384,(3,3),padding='same',activation='relu')(x)
    x=layers.Conv2D(384,(3,3),padding='same',activation='relu')(x)
    x=layers.Conv2D(256,(3,3),padding='same',activation='relu')(x)
    x=layers.MaxPooling2D((3,3),strides=2)(x)

    x=layers.Flatten()(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dense(1000,activation='relu')(x)
    y=layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="AlexNet")
    return model

In [ ]:
def AlexNet_dropout(d):
    x_input=layers.Input((227,227,3))
    x=layers.Conv2D(96,(11,11),strides=4,activation='relu')(x_input)
    x=layers.MaxPooling2D((3,3),strides=2)(x)
    x=layers.Dropout(d)(x)
    x=layers.Conv2D(256,(5,5),padding='same',activation='relu')(x)
    x=layers.MaxPooling2D((3,3),strides=2)(x)
    x=layers.Dropout(d)(x)
    x=layers.Conv2D(384,(3,3),padding='same',activation='relu')(x)
    x=layers.Conv2D(384,(3,3),padding='same',activation='relu')(x)
    x=layers.Dropout(d)(x)
    x=layers.Conv2D(256,(3,3),padding='same',activation='relu')(x)
    x=layers.MaxPooling2D((3,3),strides=2)(x)

    x=layers.Flatten()(x)
    x=layers.Dropout(d)(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dropout(d)(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dropout(d)(x)
    x=layers.Dense(1000,activation='relu')(x)
    y=layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.models.Model(inputs=x_input,outputs=y,name="AlexNet")
    return model

In [ ]:
def vgg16():
    x_input=layers.Input((224,224,3))
    x=layers.Conv2D(64,(3,3),padding="same",activation='relu')(x_input)
    x=layers.Conv2D(64,(3,3),padding="same",activation='relu')(x)
    x=layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Conv2D(128,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(128,(3,3),padding="same",activation='relu')(x)
    x=layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Conv2D(256,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(256,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(256,(3,3),padding="same",activation='relu')(x)
    x=layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.Conv2D(512,(3,3),padding="same",activation='relu')(x)
    x=layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    
    x=layers.Flatten()(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dense(4096,activation='relu')(x)
    x=layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.models.Model(inputs=x_input,outputs=x,name="Vgg16")
    return model

In [ ]:
from keras.applications.vgg16 import VGG16
import tensorflow as tf
def vgg16_pretrained(d):
    vggmodel = VGG16(weights='imagenet', include_top=True)
    for layers in (vggmodel.layers)[:19]:
        layers.trainable = False
    X=vggmodel.layers[-4].output # flatten
    X=tf.keras.layers.Dropout(d)(X)
    X=vggmodel.layers[-3](X)     # fc1
    X=tf.keras.layers.Dropout(d)(X)
    X=vggmodel.layers[-2](X)     # fc2
    X=tf.keras.layers.Dropout(d)(X)
    predictions = tf.keras.layers.Dense(1, activation="sigmoid")(X)
    model = tf.keras.models.Model(inputs = vggmodel.input, outputs = predictions)
    return model

In [ ]:
input_size=227
# model=modelv4()
# model=modelv4_overlapping_pooling()
model=modelv4_overlapping_pooling_max()
# model=AlexNet()
# model=vgg16()
# model=vgg16_pretrained(trainable=False)
model.summary()

In [ ]:
! nvidia-smi -L

In [ ]:
train_val_split=0.96
split_idx=round(train_val_split*len(train_list))
print("train samples:",len(train_list[:split_idx]))
print("validation samples:",len(train_list[split_idx:]))

In [ ]:
model_checkpoint=tf.keras.callbacks.ModelCheckpoint("cat_dog.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
lr_plateau=tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.5,patience=2,verbose=1,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['acc']) 
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['acc']) 
train_dataset=tf_dataset(train_list[:split_idx],batch_size=128,aug=True)
val_dataset=tf_dataset(train_list[split_idx:],batch_size=128)
history=model.fit(train_dataset,validation_data=val_dataset,verbose=1,epochs=5,callbacks=[model_checkpoint,lr_plateau])

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['acc'])
history=model.fit(train_dataset,validation_data=val_dataset,verbose=1,epochs=20,callbacks=[model_checkpoint,lr_plateau])

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(20,6))
fig.add_subplot(1,2,1)
plt.title("loss")
plt.plot(history.epoch,history.history['loss'],label='train_loss')
plt.plot(history.epoch,history.history['val_loss'],label='val_loss')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(loc="best")

fig.add_subplot(1,2,2)
plt.title("accuracy")
plt.plot(history.epoch,history.history['acc'],label='train_acc')
plt.plot(history.epoch,history.history['val_acc'],label='val_acc')
plt.xlabel("epochs")
plt.ylabel("acc")
plt.legend(loc="best")
plt.show()

In [ ]:
model=tf.keras.models.load_model("cat_dog.h5")

In [ ]:
def load_test_data(img_path):
    img_id=tf.strings.split(img_path,"/")[-1]
    img_id=tf.strings.split(img_id,".")[0]
    file=tf.io.read_file(img_path)
    img=tf.image.decode_image(file,expand_animations=False)
    img=tf.image.resize(img,[input_size,input_size])
    img=img/255
    return img_id,img

test_dataset=tf.data.Dataset.from_tensor_slices((test_list))
test_dataset=test_dataset.map(load_test_data)
test_dataset=test_dataset.batch(64)

In [ ]:
for data in test_dataset.take(1):
    print(data[0].shape,data[1].shape)
    print(list(map(lambda x: x.decode(),data[0].numpy())))

In [ ]:
for i,data in enumerate(test_dataset):
    test_preds= model.predict(data[1]) if i==0 else np.r_[test_preds,model.predict(data[1])]
    test_ids= data[0].numpy() if i==0 else np.r_[test_ids,data[0].numpy()]
test_ids.shape,test_preds.shape

In [ ]:
test_ids=list(map(lambda x: x.decode(),test_ids))

In [ ]:
len(test_ids),test_preds.shape

In [ ]:
if os.path.exists('submission.csv'):
    os.remove('submission.csv')
data = {'id': test_ids, 'label': test_preds[:,0]}          
df = pd.DataFrame(data)
df.to_csv('submission.csv', mode='a', index=False, header=True)

In [ ]:
def label_test_data():
    if os.path.exists('submission.csv'):
        os.remove('submission.csv')
    df=pd.DataFrame({"id":[],"label":[]})
    df.to_csv('submission.csv', mode='a', index=False, header=True)
    ids=[]
    imgs=np.array([])
    batch_size=32
    for i,test_path in enumerate(test_list):
        test_id=test_path.rsplit('/',1)[-1]
        test_id=test_id.rsplit('.',1)[0]
        img=PIL.Image.open(test_path).resize((input_size,input_size))
        img=np.array(img)
        img=img/255
        img=np.expand_dims(img,axis=0)
        
        if imgs.shape==(0,):
            imgs=img
        else:
            imgs=np.r_[imgs,img]
            
        ids.append(test_id)
        if i!=0 and (i+1)%batch_size==0:
            pred=model.predict(imgs)
            labels=pred[:,0]
#             labels=np.round(pred[:,0],2)

            data = {'id': ids, 'label': labels}  
            df = pd.DataFrame(data)
            df.to_csv('submission.csv', mode='a', index=False, header=False)
            ids=[]
            imgs=np.array([])
        if(i%1000==0):
            print(i,"out of",len(test_list))
            
    if imgs.shape!=(0,):
        pred=model.predict(imgs)
        labels=pred[:,0]
        data = {'id': ids, 'label': labels}  
        df = pd.DataFrame(data)
        df.to_csv('submission.csv', mode='a', index=False, header=False)
        ids=[]
        imgs=np.array([])
# label_test_data()

In [ ]:
df=pd.read_csv("submission.csv")
df